In [ ]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip freeze

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.8.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.6.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.3
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.7
cuda-python==12.2.1
cudf-cu12 @ https://pypi.nvidia.c

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

In [ ]:
import pandas as pd
from typing import Dict, Text, Optional, List, Any

## ini cmn merge doang

In [ ]:
import json
import os

In [ ]:
# def load_json(file_path):
#     with open(file_path, 'r') as file:
#         return json.load(file)

# def extract_item_values(data, key):
#     return [item[key] for item in data]

In [ ]:
# def merge_json_data_from_directory(directory):
#     merged_data = []
#     for filename in os.listdir(directory):
#         if filename.endswith('.json'):
#             file_path = os.path.join(directory, filename)
#             merged_data.extend(load_json(file_path))
#     return merged_data

# ta_json_directory = '/content/TA_DS_JSON'
# hotel_json_directory = '/content/HOTEL_DS_JSON'

In [ ]:
# ta_data = merge_json_data_from_directory(ta_json_directory)
# hotel_data = merge_json_data_from_directory(hotel_json_directory)

# with open('merged_ta_data.json', 'w') as ta_file:
#     json.dump(ta_data, ta_file, indent=4)

# with open('merged_hotel_data.json', 'w') as hotel_file:
#     json.dump(hotel_data, hotel_file, indent=4)

In [ ]:
with open('/content/merged_ta_data.json', 'r') as file:
    ta_data = json.load(file)

# with open('/content/mergedhotels.json', 'r') as file:
#     hotel_data = json.load(file)

## buat back end

In [ ]:
user_data = pd.read_csv('/content/user_with_preferences.csv')
hotel_data = pd.read_csv('/content/merged_hotels.csv')

# ini berubah / perlu copas
scaler = StandardScaler()
hotel_data[['rating']] = scaler.fit_transform(hotel_data[['rating']])
# sampai sini

In [ ]:
print(json.dumps(hotel_data[:5], indent=4))

In [ ]:
with open('scaled_hotel_data.json', 'w') as file:
    json.dump(hotel_data, file, indent=4)

In [ ]:
mlb = MultiLabelBinarizer()
user_data['Preferences'] = user_data['Preferences'].apply(eval)
preferences_encoded = mlb.fit_transform(user_data['Preferences'])
preferences_df = pd.DataFrame(preferences_encoded, columns=mlb.classes_)
user_data = pd.concat([user_data, preferences_df], axis=1)

In [ ]:
def calculate_middle_point(tour_interests):
    if not tour_interests:
        return {'lat': None, 'lng': None}
    lat_sum = sum(point['lat'] for point in tour_interests)
    lng_sum = sum(point['lng'] for point in tour_interests)
    middle_lat = lat_sum / len(tour_interests)
    middle_lng = lng_sum / len(tour_interests)
    return {'user_lat': middle_lat, 'user_lng': middle_lng}

In [ ]:
# misal ambil nama TA di sini

print(ta_data[:5])
print()
print(ta_data[0]['name'])
print(ta_data[1]['name'])
print(ta_data[2]['name'])
print(ta_data[3]['name'])
print(ta_data[4]['name'])

[{'formatted_address': '2965+2CR, Air Panas Baru, Air Hangat, Kerinci Regency, Jambi, Indonesia', 'name': 'Pemandangan', 'place_id': 'ChIJQSQQVgCnLS4RnABV6-AMfmw', 'rating': 2.0, 'types': "['tourist_attraction', 'point_of_interest', 'establishment']", 'user_ratings_total': 1.0, 'photos': None, 'lat': -1.9898945, 'lng': 101.3585208}, {'formatted_address': 'Jalan Wisata Air Panas Semurup No. 1, Koto Baru Semurup, Air Hangat, Air Panas Baru, Kec. Air Hangat, Kabupaten Kerinci, Jambi 37161, Indonesia', 'name': 'Air Panas Semurup', 'place_id': 'ChIJdXJozXenLS4RCQzic-sTdTE', 'rating': 4.0, 'types': "['tourist_attraction', 'point_of_interest', 'establishment']", 'user_ratings_total': 462.0, 'photos': '[{\'height\': 3120, \'html_attributions\': [\'<a href="https://maps.google.com/maps/contrib/106484250746157401405">Ardiansyah Ahmad</a>\'], \'photo_reference\': \'AUGGfZljEfmWTYe_pVNVzsh57W0f6NhhqO8IkaWatHG3mryAI_PBQzcPxoeyVHnjxesRITSAOVo9a1ovJDEkYuVgBdPZonpUCw_zeF4o7OFtxYkNAJXzeD25FfSkUSX9o8GHf

In [ ]:
# ambil bbrp TA coba

# ini berubah / perlu copas
ta_dict = {item['name']: {'lat': item['lat'], 'lng': item['lng']} for item in ta_data}
hotel_dict = {row['name']: {'lat': row['lat'], 'lng': row['lng']} for _, row in hotel_data.iterrows()}

def dptin_kordinat(name: str, data_dict: Dict[str, Dict[str, float]]) -> Optional[Dict[str, float]]:
  return data_dict.get(name)

destinasi_1 = dptin_kordinat("Air Panas Semurup", ta_dict)
destinasi_2 = dptin_kordinat("Tebat Air Koto Majidin", ta_dict)
destinasi_3 = dptin_kordinat("Air Terjun Pendung Mudik", ta_dict)
destinasi_4 = dptin_kordinat("Kebun jeruk arumi&hanum", ta_dict)

tour_interests = [destinasi_1, destinasi_2, destinasi_3, destinasi_4]
# sampai sini

In [ ]:
middle_point = calculate_middle_point(tour_interests)
print(f"Middle Point: {middle_point}")

Middle Point: {'user_lat': -1.9904434000000002, 'user_lng': 101.37182865}


In [ ]:
# middle_point = {'user_lat': -6.487506666666666, 'user_lng': 106.84090666666667}
# -6.4454293920692765, 106.86835709915543 rumah mat
# -6.487506666666666, 106.84090666666667

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance



# setiap get lat/lng yg baru perlu run ginian
# ini berubah / perlu copas
def distances(middle_point: Dict[str, float], data: pd.DataFrame) -> pd.DataFrame:
    data['distance'] = data.apply(lambda row: haversine(middle_point['user_lat'], middle_point['user_lng'], row['lat'], row['lng']), axis=1)
    return data

def normalize_distances(data: pd.DataFrame) -> pd.DataFrame:
    distances = data['distance'].values.reshape(-1, 1)
    scaler = StandardScaler()
    data['distance_normalized'] = scaler.fit_transform(distances)
    return data

hotel_data = distances(middle_point, hotel_data)
hotel_data = normalize_distances(hotel_data)
# sampai sini

# hotel_data['distance'] = hotel_data.apply(lambda row: haversine(middle_point['user_lat'], middle_point['user_lng'], row['lat'], row['lng']), axis=1)
# hotel_data['distance_normalized'] = scaler.fit_transform(hotel_data[['distance']])

In [ ]:
class RankingModel(tfrs.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        unique_hotel_names = hotel_data["name"].astype(str).unique()

        self.hotel_embeddings = tf.keras.Sequential([
            tf.keras.layers.StringLookup(vocabulary=unique_hotel_names, mask_token=None),
            tf.keras.layers.Embedding(len(unique_hotel_names) + 1, embedding_dimension)
        ])

        # ini berubah / perlu copas
        self.user_preference_embeddings = tf.keras.layers.Embedding(len(mlb.classes_) + 1, embedding_dimension)

        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(1)
        ])

        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        hotel_embeddings = self.hotel_embeddings(features["hotel_name"])
        distance_normalized_expanded = tf.expand_dims(features["distance_normalized"], axis=1)
        rating_expanded = tf.expand_dims(features["rating"], axis=1)
        user_preferences = tf.reduce_sum(self.user_preference_embeddings(features["preferences"]), axis=1)
        concatenated = tf.concat([hotel_embeddings, distance_normalized_expanded, rating_expanded, user_preferences], axis=1)
        return self.rating_model(concatenated)

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        labels = features.pop("ranking_score")
        predictions = self(features)
        return self.task(labels=labels, predictions=predictions)

In [ ]:
from collections import Counter

# Extract hotel names
hotel_names = [hotel_data["name"] for hotel in hotel_data]

# Find duplicates
duplicate_names = [item for item, count in Counter(hotel_names).items() if count > 1]

print(f"Duplicate hotel names: {duplicate_names}")

In [ ]:
def ensure_unique_names(data):
    name_count = Counter(hotel["name"] for hotel in data)
    unique_data = []
    seen = {}
    for hotel in data:
        name = hotel["name"]
        if name_count[name] > 1:
            if name not in seen:
                seen[name] = 1
            else:
                seen[name] += 1
            unique_name = f"{name}_{seen[name]}"
            hotel["name"] = unique_name
        unique_data.append(hotel)

    return unique_data

hotel_data_baru = ensure_unique_names(hotel_data)

In [ ]:
print(json.dumps(hotel_data_baru[-50:], indent=2))

In [ ]:
with open('model_architecture.json', 'r') as f:
    model_json = f.read()
    model = tf.keras.models.model_from_json(model_json, custom_objects={'RankingModel': RankingModel})

In [ ]:
model = RankingModel()

dummy_input = {
    "hotel_name": tf.constant(["dummy_hotel"]),
    "distance_normalized": tf.constant([-0.041947]),
    "rating": tf.constant([4.0]),
    "preferences": tf.constant([[0, 1, 0, 1, 0,
                                 1, 0, 0, 0, 0,
                                 0, 0, 0, 0, 0,
                                 0, 0, 1]])
}
_ = model(dummy_input)


In [ ]:
model.load_weights('model_weights_stdrecommend.h5')

ValueError: Cannot assign value to variable ' embedding_2/embeddings:0': Shape mismatch.The variable shape (20831, 32), and the assigned value shape (231, 32) are incompatible.

In [ ]:
def rank_hotels(user_id, top_n):
    user_preferences = user_data.loc[user_data['User_Id'] == user_id, mlb.classes_].values.flatten()
    print(user_preferences)
    hotel_data = {
        "hotel_name": tf.constant(hotel_data["name"].tolist()),
        "distance_normalized": tf.constant(hotel_data["distance_normalized"].tolist(), dtype=tf.float32),
        "rating": tf.constant(hotel_data["rating"].tolist(), dtype=tf.float32),
        "preferences": tf.constant([user_preferences] * len(hotel_data), dtype=tf.float32)
    }
    predictions = model(hotel_data)
    hotel_data["predicted_ranking_score"] = predictions.numpy().flatten()
    ranked_hotels = hotel_data.sort_values(by="predicted_ranking_score", ascending=False).head(top_n)
    return ranked_hotels[["name", "formatted_address", "distance", "rating", "predicted_ranking_score", "photos"]]

In [ ]:
print(middle_point)
user_id = 1
top_hotels = rank_hotels(user_id, top_n=15)
print(top_hotels)